# Imports

In [1]:
import numpy as np

# Add these lines if you clone from https://github.com/fdalvi/NeuroX
# import sys
# sys.path.append("/path/to/NeuroX")
# In the case of the notebook, the path is the parent directory
import sys
sys.path.append("..")

# Data

In [2]:
!cat test.in

This is a test sentence .
This is another sentence .
The NeuroX toolkit can be used for neuron analysis !
This sentence contains an <<UNKNOWN>> word .
Individual neuron activations can be insightful .
It is a beautiful day outside .
The red car zoomed by quickly !
Is this a question ?
There are more than 10000 neurons in some models .
Hope you are enjoying the tutorial !

In [3]:
!cat test.label

DT VBZ DT NN NN .
DT VBZ DT NN .
DT NNP NN MD VB VBN IN JJ NN .
DT NN VBZ DT NN NN .
NNP CC NNS MD VB JJ .
PRP VBZ DT JJ NN IN .
DT JJ NN VBN IN RB .
VBZ DT DT NN .
EX VBP JJR IN CD NNS IN DT NNS .
NN PRP VBP VBG DT NN .

# Extract Representations

In [4]:
import neurox.data.extraction.transformers_extractor as transformers_extractor
transformers_extractor.extract_representations('bert-base-uncased',
    'test.in',
    'activations.json',
    aggregation="average" #last, first
)

Loading model: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Reading input corpus
Preparing output file
Extracting representations from model
Sentence         : "This is a test sentence ."
Original    (006): ['This', 'is', 'a', 'test', 'sentence', '.']
Tokenized   (008): ['[CLS]', 'this', 'is', 'a', 'test', 'sentence', '.', '[SEP]']
Filtered   (006): ['this', 'is', 'a', 'test', 'sentence', '.']
Detokenized (006): ['this', 'is', 'a', 'test', 'sentence', '.']
Counter: 6
Hidden states:  (13, 6, 768)
# Extracted words:  6
Sentence         : "This is another sentence ."
Original    (005): ['This', 'is', 'another', 'sentence', '.']
Tokenized   (007): ['[CLS]', 'this', 'is', 'another', 'sentence', '.', '[SEP]']
Filtered   (005): ['this', 'is', 'another', 'sentence', '.']
Detokenized (005): ['this', 'is', 'another', 'sentence', '.']
Counter: 5
Hidden states:  (13, 5, 768)
# Extracted words:  5
Sentence         : "The NeuroX toolkit can be used for neuron analysis !"
Original    (010): ['The', 'NeuroX', 'toolkit', 'can', 'be', 'used', 'for', 'neuron', 'a

In [5]:
!cut -c1-120 < activations.json

{"linex_index": 0, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.67392403, -0.09324985
{"linex_index": 1, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.67392403, -0.09324985
{"linex_index": 2, "features": [{"token": "the", "layers": [{"index": 0, "values": [-0.43669653, 0.53601718, -0.05141334
{"linex_index": 3, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.67392403, -0.09324985
{"linex_index": 4, "features": [{"token": "individual", "layers": [{"index": 0, "values": [0.14265811, 0.75676745, -0.91
{"linex_index": 5, "features": [{"token": "it", "layers": [{"index": 0, "values": [-0.42272022, -0.02889436, -0.14556105
{"linex_index": 6, "features": [{"token": "the", "layers": [{"index": 0, "values": [-0.43669653, 0.53601718, -0.05141334
{"linex_index": 7, "features": [{"token": "is", "layers": [{"index": 0, "values": [-0.29041198, 0.00098542, -0.43960977,
{"linex_index": 8, "features": [

# Prepare Data

In [6]:
import neurox.data.loader as data_loader
activations, num_layers = data_loader.load_activations('activations.json', 768)

Loading json activations from activations.json...
10 13.0


In [7]:
print(activations)

[array([[-0.6485135 ,  0.67392403, -0.09324985, ..., -0.5617491 ,
         0.48687834,  0.54459047],
       [-0.62703037, -0.06331295, -0.31427881, ..., -0.15095142,
         0.20867236,  0.97557974],
       [ 0.39637804,  0.31565616,  0.02008923, ..., -0.07199516,
         0.2717956 ,  1.4120574 ],
       [ 0.60103226, -0.69701707, -0.2000732 , ..., -0.72475618,
         0.08129568, -0.25088236],
       [ 0.83230692,  0.28782725,  0.00214796, ..., -0.11331076,
        -0.15800183, -0.34240773],
       [-0.078184  ,  0.01739771, -0.28644535, ...,  0.20194323,
        -0.62845349, -0.45307434]]), array([[-0.6485135 ,  0.67392403, -0.09324985, ..., -0.1489853 ,
         0.25174388,  0.49430278],
       [-0.62703037, -0.06331295, -0.31427881, ..., -0.08159983,
         0.22553164,  1.23856652],
       [ 0.17470703,  0.5296641 ,  0.15547034, ..., -0.02144399,
         0.16490568,  0.45266795],
       [ 0.79593539,  0.43478003,  0.19534861, ...,  0.24064474,
        -0.1610865 , -0.04338051

In [8]:
activations[0].shape # 9984 -> 768 * 13

(6, 9984)

In [9]:
activations[1].shape

(5, 9984)

In [10]:
# load_data also does sanity checks for parallelism between tokens, labels and activations
tokens = data_loader.load_data('test.in',
                               'test.label',
                               activations,
                               512 # max_sent_l
                              )

In [11]:
tokens

{'source': [['This', 'is', 'a', 'test', 'sentence', '.'],
  ['This', 'is', 'another', 'sentence', '.'],
  ['The',
   'NeuroX',
   'toolkit',
   'can',
   'be',
   'used',
   'for',
   'neuron',
   'analysis',
   '!'],
  ['This', 'sentence', 'contains', 'an', '<<UNKNOWN>>', 'word', '.'],
  ['Individual', 'neuron', 'activations', 'can', 'be', 'insightful', '.'],
  ['It', 'is', 'a', 'beautiful', 'day', 'outside', '.'],
  ['The', 'red', 'car', 'zoomed', 'by', 'quickly', '!'],
  ['Is', 'this', 'a', 'question', '?'],
  ['There',
   'are',
   'more',
   'than',
   '10000',
   'neurons',
   'in',
   'some',
   'models',
   '.'],
  ['Hope', 'you', 'are', 'enjoying', 'the', 'tutorial', '!']],
 'target': [['DT', 'VBZ', 'DT', 'NN', 'NN', '.'],
  ['DT', 'VBZ', 'DT', 'NN', '.'],
  ['DT', 'NNP', 'NN', 'MD', 'VB', 'VBN', 'IN', 'JJ', 'NN', '.'],
  ['DT', 'NN', 'VBZ', 'DT', 'NN', 'NN', '.'],
  ['NNP', 'CC', 'NNS', 'MD', 'VB', 'JJ', '.'],
  ['PRP', 'VBZ', 'DT', 'JJ', 'NN', 'IN', '.'],
  ['DT', 'JJ', 'NN'

In [12]:
import neurox.interpretation.utils as utils
X, y, mapping = utils.create_tensors(tokens, activations, 'NN')
label2idx, idx2label, src2idx, idx2src = mapping

Number of tokens:  71
length of source dictionary:  51
length of target dictionary:  19
71
Total instances: 71
['another', 'toolkit', 'red', '?', 'used', 'beautiful', 'this', 'Is', 'NeuroX', 'are', '<<UNKNOWN>>', 'Individual', 'can', 'neurons', 'tutorial', '.', 'enjoying', 'outside', 'in', 'for']


In [13]:
# X_dev, y_dev, mapping = utils.create_tensors(dev_tokens, dev_activations, 'LABEL1', mapping=mapping)
X[0] #9984

array([-0.6485135 ,  0.673924  , -0.09324985, ..., -0.5617491 ,
        0.48687834,  0.5445905 ], dtype=float32)

# Train Probing Classifier

In [14]:
import neurox.interpretation.linear_probe as linear_probe
probe = linear_probe.train_logistic_regression_probe(X, y, lambda_l1=0.001, lambda_l2=0.001)

Training classification probe
Creating model...
Number of training instances: 71
Number of classes: 19


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.1473


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0452


...

Epoch: [8/10], Loss: 0.0377


epoch [9/10]: 0it [00:00, ?it/s]

Epoch: [9/10], Loss: 0.0357


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0336


# Evaluate Model

In [15]:
linear_probe.evaluate_probe(probe, X, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 1.00


{'__OVERALL__': 1.0,
 'JJ': 1.0,
 'VBG': 1.0,
 'VB': 1.0,
 'NNP': 1.0,
 'MD': 1.0,
 'EX': 1.0,
 'VBP': 1.0,
 '.': 1.0,
 'NN': 1.0,
 'VBZ': 1.0,
 'VBN': 1.0,
 'NNS': 1.0,
 'CC': 1.0,
 'CD': 1.0,
 'RB': 1.0,
 'JJR': 1.0,
 'IN': 1.0,
 'DT': 1.0,
 'PRP': 1.0}

# Layerwise probing

In [16]:
# [layer0_neuron0, layer0_neuron1, layer0_neuron2 ...., layer1_neuron0, layer1_neuron1,....., layer13_neuron768]
# [768,768,770...1536] <- for layer 1
import neurox.interpretation.ablation as ablation

In [17]:
layer_0_X = ablation.filter_activations_by_layers(X, [0], 13)
print(layer_0_X.shape)
probe_layer_0 = linear_probe.train_logistic_regression_probe(layer_0_X, y, lambda_l1=0.001, lambda_l2=0.001)
linear_probe.evaluate_probe(probe_layer_0, layer_0_X, y, idx_to_class=idx2label)

(71, 768)
Training classification probe
Creating model...
Number of training instances: 71
Number of classes: 19


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.1301


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.1067


...

epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0285


Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 1.00


{'__OVERALL__': 1.0,
 'JJ': 1.0,
 'VBG': 1.0,
 'VB': 1.0,
 'NNP': 1.0,
 'MD': 1.0,
 'EX': 1.0,
 'VBP': 1.0,
 '.': 1.0,
 'NN': 1.0,
 'VBZ': 1.0,
 'VBN': 1.0,
 'NNS': 1.0,
 'CC': 1.0,
 'CD': 1.0,
 'RB': 1.0,
 'JJR': 1.0,
 'IN': 1.0,
 'DT': 1.0,
 'PRP': 1.0}

# Get Neuron Ranking

In [18]:
ordering, cutoffs = linear_probe.get_neuron_ordering(probe, label2idx)

  0%|          | 0/101 [00:00<?, ?it/s]

In [19]:
ordering[:10]

[7552, 5714, 6227, 6683, 1436, 1757, 2336, 2979, 4134, 4902]

# Train on top N neurons

In [20]:
X_selected = ablation.filter_activations_keep_neurons(X, ordering[:100])

In [21]:
X_selected.shape

(71, 100)

In [22]:
probe_selected = linear_probe.train_logistic_regression_probe(X_selected, y, lambda_l1=0.001, lambda_l2=0.001)

Training classification probe
Creating model...
Number of training instances: 71
Number of classes: 19


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.1307


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.1255


...

Epoch: [8/10], Loss: 0.1008


epoch [9/10]: 0it [00:00, ?it/s]

Epoch: [9/10], Loss: 0.0974


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0942


In [23]:
linear_probe.evaluate_probe(probe_selected, X_selected, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.42


{'__OVERALL__': 0.4225352112676056,
 'JJ': 0.0,
 'VBG': 0.0,
 'VB': 0.0,
 'NNP': 0.0,
 'MD': 0.0,
 'EX': 0.0,
 'VBP': 0.0,
 '.': 1.0,
 'NN': 0.38461538461538464,
 'VBZ': 0.2,
 'VBN': 0.0,
 'NNS': 0.0,
 'CC': 0.0,
 'CD': 0.0,
 'RB': 0.0,
 'JJR': 0.0,
 'IN': 0.0,
 'DT': 1.0,
 'PRP': 0.5}

# Further analysis

In [24]:
top_neurons, top_neurons_per_class = linear_probe.get_top_neurons(probe, 0.01, label2idx)
print(top_neurons)

[8202 6159 4112 6164 8215 2076   38 4134 6194 2104 6204 2108 8255 8256
 4164 8261 8262   76 2128 6227   87 2136 8284 2144   99 4204 2160 2167
 8320 2185 6282 8330  140 6285 8337 4242 4247 2203 6302 6308 2212 8356
 6313 8362 8371 2227 6329 8380 4293 2250 2252 4302 2259  213 2269 2273
  228  250 2299 4361 2313 6418 8466 2329 2336  292 4390  302  308 6452
 4407 2359  314 2376 6472 6476 6483 4439 8553 6508 4461 4464 4465 2423
 2429 2439 8594 8596 4508 6559 6560 4515 6568 6572 2486  442 8637 6597
 6604 4565 6614 2531 6627 8683 4592 6642 6643 6647 8699  508 8712 6683
 8737 4644  553 4650 2609 4658 4660 2612 8756 4666 8768 8789 6741 2647
 2650  617 8812 8815 6769 2677 4727 4739 8853 4759 4776 6837 8888 4798
 8896 2758 8908 4816 6867 6876 4842 4846  764  778 8970 4878 8976 4892
 6940 2846  806 4902 4916 2870 4921 2875 2876 9022  831 2883 4932  838
  839  840 9033 4935 2891 4946 2908 2912 2920 7021 7026 4985 7034 4991
 7045 9098 5003 7050 2960 2961 7060 2965 9109 2964 2972 2979 2981 5031
 9128 

In [25]:
X_zeroed = ablation.zero_out_activations_keep_neurons(X, top_neurons)
linear_probe.evaluate_probe(probe, X_zeroed, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.83


{'__OVERALL__': 0.8309859154929577,
 'JJ': 1.0,
 'VBG': 1.0,
 'VB': 0.0,
 'NNP': 0.0,
 'MD': 0.0,
 'EX': 1.0,
 'VBP': 1.0,
 '.': 1.0,
 'NN': 1.0,
 'VBZ': 0.8,
 'VBN': 0.0,
 'NNS': 1.0,
 'CC': 1.0,
 'CD': 0.0,
 'RB': 0.0,
 'JJR': 0.0,
 'IN': 1.0,
 'DT': 1.0,
 'PRP': 1.0}

In [26]:
import neurox.analysis.visualization as visualization

visualization.visualize_activations(
    " ".join(tokens["source"][9]),
    activations[9][:, 2]
)

In [27]:
# Visualize entire dataset
from neurox.data.extraction.transformers_extractor import get_model_and_tokenizer, extract_sentence_representations

from IPython.display import display, SVG
class Viz:
    def __init__(self, model_name):
        self.model_name = model_name
        self.model, self.tokenizer = get_model_and_tokenizer(model_name)

    def __call__(self, sentence, layer, neuron):
        a, _ = extract_sentence_representations(sentence, self.model, self.tokenizer)
        return SVG(visualization.visualize_activations(sentence, a[layer, :, neuron]).tostring())

viz = Viz('bert-base-uncased')
for s_idx in range(len(tokens["source"])):
    display(viz(" ".join(tokens["source"][s_idx]), 0, 5))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sentence         : "This is a test sentence ."
Original    (006): ['This', 'is', 'a', 'test', 'sentence', '.']
Tokenized   (008): ['[CLS]', 'this', 'is', 'a', 'test', 'sentence', '.', '[SEP]']
Filtered   (006): ['this', 'is', 'a', 'test', 'sentence', '.']
Detokenized (006): ['this', 'is', 'a', 'test', 'sentence', '.']
Counter: 6


Sentence         : "This is another sentence ."
Original    (005): ['This', 'is', 'another', 'sentence', '.']
Tokenized   (007): ['[CLS]', 'this', 'is', 'another', 'sentence', '.', '[SEP]']
Filtered   (005): ['this', 'is', 'another', 'sentence', '.']
Detokenized (005): ['this', 'is', 'another', 'sentence', '.']
Counter: 5


Sentence         : "The NeuroX toolkit can be used for neuron analysis !"
Original    (010): ['The', 'NeuroX', 'toolkit', 'can', 'be', 'used', 'for', 'neuron', 'analysis', '!']
Tokenized   (016): ['[CLS]', 'the', 'ne', '##uro', '##x', 'tool', '##kit', 'can', 'be', 'used', 'for', 'ne', '##uron', 'analysis', '!', '[SEP]']
Filtered   (014): ['the', 'ne', '##uro', '##x', 'tool', '##kit', 'can', 'be', 'used', 'for', 'ne', '##uron', 'analysis', '!']
Detokenized (010): ['the', 'ne##uro##x', 'tool##kit', 'can', 'be', 'used', 'for', 'ne##uron', 'analysis', '!']
Counter: 14


Sentence         : "This sentence contains an <<UNKNOWN>> word ."
Original    (007): ['This', 'sentence', 'contains', 'an', '<<UNKNOWN>>', 'word', '.']
Tokenized   (013): ['[CLS]', 'this', 'sentence', 'contains', 'an', '<', '<', 'unknown', '>', '>', 'word', '.', '[SEP]']
Filtered   (011): ['this', 'sentence', 'contains', 'an', '<', '<', 'unknown', '>', '>', 'word', '.']
Detokenized (007): ['this', 'sentence', 'contains', 'an', '<<unknown>>', 'word', '.']
Counter: 11


...

Sentence         : "It is a beautiful day outside ."
Original    (007): ['It', 'is', 'a', 'beautiful', 'day', 'outside', '.']
Tokenized   (009): ['[CLS]', 'it', 'is', 'a', 'beautiful', 'day', 'outside', '.', '[SEP]']
Filtered   (007): ['it', 'is', 'a', 'beautiful', 'day', 'outside', '.']
Detokenized (007): ['it', 'is', 'a', 'beautiful', 'day', 'outside', '.']
Counter: 7


Sentence         : "The red car zoomed by quickly !"
Original    (007): ['The', 'red', 'car', 'zoomed', 'by', 'quickly', '!']
Tokenized   (010): ['[CLS]', 'the', 'red', 'car', 'zoom', '##ed', 'by', 'quickly', '!', '[SEP]']
Filtered   (008): ['the', 'red', 'car', 'zoom', '##ed', 'by', 'quickly', '!']
Detokenized (007): ['the', 'red', 'car', 'zoom##ed', 'by', 'quickly', '!']
Counter: 8


Sentence         : "Is this a question ?"
Original    (005): ['Is', 'this', 'a', 'question', '?']
Tokenized   (007): ['[CLS]', 'is', 'this', 'a', 'question', '?', '[SEP]']
Filtered   (005): ['is', 'this', 'a', 'question', '?']
Detokenized (005): ['is', 'this', 'a', 'question', '?']
Counter: 5


Sentence         : "There are more than 10000 neurons in some models ."
Original    (010): ['There', 'are', 'more', 'than', '10000', 'neurons', 'in', 'some', 'models', '.']
Tokenized   (013): ['[CLS]', 'there', 'are', 'more', 'than', '1000', '##0', 'neurons', 'in', 'some', 'models', '.', '[SEP]']
Filtered   (011): ['there', 'are', 'more', 'than', '1000', '##0', 'neurons', 'in', 'some', 'models', '.']
Detokenized (010): ['there', 'are', 'more', 'than', '1000##0', 'neurons', 'in', 'some', 'models', '.']
Counter: 11


Sentence         : "Hope you are enjoying the tutorial !"
Original    (007): ['Hope', 'you', 'are', 'enjoying', 'the', 'tutorial', '!']
Tokenized   (010): ['[CLS]', 'hope', 'you', 'are', 'enjoying', 'the', 'tutor', '##ial', '!', '[SEP]']
Filtered   (008): ['hope', 'you', 'are', 'enjoying', 'the', 'tutor', '##ial', '!']
Detokenized (007): ['hope', 'you', 'are', 'enjoying', 'the', 'tutor##ial', '!']
Counter: 8
